In [17]:
import pandas as pd
import numpy as np

# Import Data

In [18]:
df_gravimetri = pd.read_csv('hasil/ET Aktual Gravimetri.csv')
df_bc = pd.read_csv('hasil/Estimasi ET Blaney-Criddle.csv')
df_hs = pd.read_csv('hasil/Estimasi ET Hargreaves-Samani.csv')
df_pm = pd.read_csv('hasil/Estimasi ET Penman-Monteith.csv')

In [19]:
df_gravimetri.head(3)

,A1,A2,A3,A4,B1,B2,B3,B4,C1,C2,...,D3,D4,E1,E2,E3,E4,F1,F2,F3,F4
0,2.56,3.072,2.656,6.592,0.0,0.0,0.0,0.0,3.248980,7.395918,...,7.5250,7.95,7.575309,8.148148,13.066667,7.664198,23.984,13.456,15.848,9.880
1,45.92,21.120,6.464,44.000,0.0,0.0,0.0,0.0,18.269388,11.608163,...,6.0125,1.80,4.414815,5.392593,9.580247,12.898765,32.240,4.824,0.456,73.768
2,0.00,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.000


In [20]:
df_bc.head(3)

,Day,Tanggal,Mean Suhu Udara,ET Total
0,day 1,2024-10-22,30.01500,5.887863
1,day 2,2024-10-23,30.94875,6.003835
2,day 3,2024-10-24,31.14000,6.027588


In [21]:
df_hs.head(3)

,Day,Tanggal,Max Suhu Udara,Min Suhu Udara,Mean Suhu Udara,Radiasi Extrateresterial,ET Total
0,day 1,2024-10-22,32.36,24.94,28.650,15.761121,4.586724
1,day 2,2024-10-23,33.47,25.65,29.560,15.769790,4.803622
2,day 3,2024-10-24,33.64,25.69,29.665,15.777863,4.856608


In [22]:
df_pm.head(3)

,Day,Tanggal,Max Suhu Udara,Min Suhu Udara,Mean Suhu Udara,Mean Kecepatan Angin,Mean Kelembapan Relatif,Radiasi Extrateresterial,ET Total
0,day 1,2024-10-22,32.36,24.94,28.650,3.06625,50.91375,15.761121,3.457631
1,day 2,2024-10-23,33.47,25.65,29.560,3.07750,47.54000,15.769790,3.797029
2,day 3,2024-10-24,33.64,25.69,29.665,2.41000,47.71875,15.777863,3.387418


# Akurasi

# Mean Absolute Error (MAE)

# Root Mean Square Error (RMSE)

# Mean Bias Error (MBE)